In this notebook, we test out hyperparameter tuning with Optuna. again. this time with just GNBlocks

In [1]:
# ipython extension to autoreload imported modules so that any changes will be up to date before running code in this nb
%load_ext autoreload 
%autoreload 2

In [2]:
from utils.jraph_training import train_and_evaluate_with_data, create_dataset
# from utils.jraph_models import MLPGraphNetwork
import ml_collections
import optuna 
from functools import partial
from datetime import datetime
import os 

/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# set up logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

### set up functions for optuna

In [4]:
CHECKPOINT_PATH = "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/experiments/tuning"

In [5]:
def objective(trial, datasets):
    """ Defines the objective function to be optimized over, aka the validation loss of a model.
    
        Args:
            trial: object which characterizes the current run 
            datasets: dictionary of data. we explicitly pass this in so that we don't have to waste runtime regenerating the same dataset over and over. 
    """
    # create config 
    config = ml_collections.ConfigDict()

    # Optimizer.
    config.optimizer = "adam"
    # config.optimizer = trial.suggest_categorical("optimizer", ["adam", "sgd"])
    config.learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, 
                                               log=True)
    if config.optimizer == "sgd":
        config.momentum = trial.suggest_float('momentum', 0, 0.999) # upper bound is inclusive, and we want to exclude a momentum of 1 because that would yield no decay 

    # Data params that are used in training 
    config.output_steps = 4

    # Training hyperparameters.
    config.batch_size = 1 # variable currently not used
    config.epochs = 5
    config.log_every_epochs = 5
    config.eval_every_epochs = 5
    config.checkpoint_every_epochs = 10

    # GNN hyperparameters.
    config.model = 'MLPBlock'
    config.dropout_rate = trial.suggest_float('dropout_rate', 0, 0.6)
    config.skip_connections = False # This was throwing a broadcast error in add_graphs_tuples_nodes when this was set to True
    config.layer_norm = False # TODO perhaps we want to turn on later

    # choose the hidden layer feature size using powers of 2 
    config.edge_features = (
        2**trial.suggest_int("edge_mlp_1_power", 1, 5), # range 2 - 64; upper bound is inclusive
        2**trial.suggest_int("edge_mlp_2_power", 1, 5), # range 2 - 64
    )
    config.node_features = (
        2**trial.suggest_int("node_mlp_1_power", 1, 8), # range 2 - 512
        2**trial.suggest_int("node_mlp_2_power", 1, 8), # range 2 - 512
        2) 
    # note the last feature size will be the number of features that the graph predicts
    config.global_features = None

    # generate a workdir 
    # TODO: check if we actually care about referencing this in the future or if we can just create a temp dir 
    workdir=os.path.join(CHECKPOINT_PATH, "checkpoints", str(datetime.now()))

    # run training 
    state, train_metrics, eval_metrics_dict = train_and_evaluate_with_data(config=config, workdir=workdir, datasets=datasets)
    
    # retrieve and return val loss (MSE)
    print("eval_metrics_dict['val'].loss", eval_metrics_dict['val'].loss)
    print()
    return eval_metrics_dict['val'].loss.total




In [6]:
def get_data_config():
    config = ml_collections.ConfigDict()

    config.n_samples=100
    config.input_steps=1
    config.output_delay=8 # predict 24 hrs into the future 
    config.output_steps=4
    config.timestep_duration=3 # equivalent to 3 hours
    # note a 3 hour timestep resolution would be 5*24/3=40
    # if the time_resolution is 120, then a sampling frequency of 3 would achieve a 3 hour timestep 
    config.sample_buffer = -1 * (config.input_steps + config.output_delay + config.output_steps - 1) # negative buffer so that our sample input are continuous (i.e. the first sample would overlap a bit with consecutive samples) 
        # number of timesteps strictly between the end 
        # of one full sample and the start of the next sample
    config.time_resolution=120 # the number of 
                # raw data points generated per time unit, equivalent to the 
                # number of data points generated per 5 days in the simulation
    config.init_buffer_samples=100
    config.train_pct=0.7
    config.val_pct=0.2
    config.test_pct=0.1
    config.K=36
    config.F=8
    config.c=10
    config.b=10
    config.h=1
    config.seed=42
    config.normalize=True

    return config

In [7]:
def prepare_study(study_name):
    # generate dataset 
    dataset_config = get_data_config()
    datasets = create_dataset(dataset_config)

    # get the objective function that reuses the pre-generated datasets 
    objective_with_dataset = partial(objective, datasets=datasets)

    # run optimization study
    db_path = os.path.join(CHECKPOINT_PATH, study_name, "optuna_hparam_search.db")
    if not os.path.exists(os.path.join(CHECKPOINT_PATH, study_name)):
        os.makedirs(os.path.join(CHECKPOINT_PATH, study_name))

    study = optuna.create_study(
        study_name=study_name,
        storage=f'sqlite:///{db_path}', # generates a new db if it doesn't exist
        direction='minimize',
        pruner=optuna.pruners.MedianPruner(
            n_startup_trials=5, 
            n_warmup_steps=3
            ), 
        load_if_exists=True, 
    )
    # study.optimize(objective_with_dataset, 
    #                n_trials=25-len(study.trials), 
    #                n_jobs=1)
    
    return study, objective_with_dataset

### try hyperparameter tuning again with fewer params and lower learning rate options

In [8]:
# get study
study3, objective_with_dataset = prepare_study(study_name="hparam_study_3")

[I 2023-11-30 23:11:42,004] A new study created in RDB with name: hparam_study_3


In [9]:
study3.optimize(objective_with_dataset, 
                n_trials=5-len(study3.trials), 
                n_jobs=1)

[I 2023-11-30 23:11:51,763] Trial 0 finished with value: 22.660564422607422 and parameters: {'learning_rate': 0.0020041930977819688, 'dropout_rate': 0.3394749796550142, 'edge_mlp_1_power': 5, 'edge_mlp_2_power': 3, 'node_mlp_1_power': 4, 'node_mlp_2_power': 1}. Best is trial 0 with value: 22.660564422607422.


eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(22.660564, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:12:00,795] Trial 1 finished with value: 35.973567962646484 and parameters: {'learning_rate': 0.0002007623077592412, 'dropout_rate': 0.06676328337052558, '

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(35.973568, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:12:09,571] Trial 2 finished with value: 22.660564422607422 and parameters: {'learning_rate': 0.000398748426518406, 'dropout_rate': 0.3588073389116452, 'ed

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(22.660564, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:12:17,697] Trial 3 finished with value: 26.933732986450195 and parameters: {'learning_rate': 0.0016599920633295026, 'dropout_rate': 0.3707390553481949, 'e

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(26.933733, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:12:25,663] Trial 4 finished with value: 22.660564422607422 and parameters: {'learning_rate': 0.00541641344004353, 'dropout_rate': 0.07740519930057081, 'ed

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(22.660564, dtype=float32), count=Array(20., dtype=float32))



ok let's try visualizing this to see what in the hell is going wrong

In [10]:
fig = optuna.visualization.plot_intermediate_values(study3)
fig.show()

[W 2023-11-30 23:12:25,944] You need to set up the pruning feature to utilize `plot_intermediate_values()`


In [11]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study3, params=['learning_rate', 'dropout_rate'])
fig.show()

In [12]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study3, params=['learning_rate', 'node_mlp_1_power'])
fig.show()

In [13]:
study3.optimize(objective_with_dataset, 
                n_trials=30-len(study3.trials), 
                n_jobs=1)

ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:12:34,409] Trial 5 finished with value: 22.660564422607422 and parameters: {'learning_rate': 0.0019121551576760103, 'dropout_rate': 0.18837785436354892, '

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(22.660564, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:12:42,182] Trial 6 finished with value: 22.660564422607422 and parameters: {'learning_rate': 0.0019316679694575728, 'dropout_rate': 0.35879899733771925, '

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(22.660564, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:12:50,558] Trial 7 finished with value: 114.60284423828125 and parameters: {'learning_rate': 0.0002911003787532303, 'dropout_rate': 0.04232998114633357, '

...

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(22.660564, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:15:53,513] Trial 28 finished with value: 22.660564422607422 and parameters: {'learning_rate': 0.0038179439176016837, 'dropout_rate': 0.3325437795339146, '

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(22.660564, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:16:01,136] Trial 29 finished with value: 35.561161041259766 and parameters: {'learning_rate': 0.001776180696293107, 'dropout_rate': 0.23418325974259002, '

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(35.56116, dtype=float32), count=Array(20., dtype=float32))



In [14]:
fig = optuna.visualization.plot_intermediate_values(study3)
fig.show()

[W 2023-11-30 23:16:01,289] You need to set up the pruning feature to utilize `plot_intermediate_values()`


In [15]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study3, params=['learning_rate', 'dropout_rate'])
fig.show()

In [16]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study3, params=['learning_rate', 'node_mlp_1_power'])
fig.show()

In [17]:
print(study3.direction)

StudyDirection.MINIMIZE


In [18]:
study3.trials

[FrozenTrial(number=0, state=TrialState.COMPLETE, values=[22.660564422607422], datetime_start=datetime.datetime(2023, 11, 30, 23, 11, 42, 127813), datetime_complete=datetime.datetime(2023, 11, 30, 23, 11, 51, 738453), params={'learning_rate': 0.0020041930977819688, 'dropout_rate': 0.3394749796550142, 'edge_mlp_1_power': 5, 'edge_mlp_2_power': 3, 'node_mlp_1_power': 4, 'node_mlp_2_power': 1}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.01, log=True, low=0.0001, step=None), 'dropout_rate': FloatDistribution(high=0.6, log=False, low=0.0, step=None), 'edge_mlp_1_power': IntDistribution(high=5, log=False, low=1, step=1), 'edge_mlp_2_power': IntDistribution(high=5, log=False, low=1, step=1), 'node_mlp_1_power': IntDistribution(high=8, log=False, low=1, step=1), 'node_mlp_2_power': IntDistribution(high=8, log=False, low=1, step=1)}, trial_id=1, value=None),
 FrozenTrial(number=1, state=TrialState.COMPLETE, values=[35.9735679